In [73]:
import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import re
import pandas as pd

In [74]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

In [75]:
my_text = extract_text_from_pdf('AnnualSalaryReport2018-2019.pdf')

In [76]:
# Grab COLUMBIA First
last = my_text[36:].split('COLUM')[-1]
columbia = my_text[36:].split('COLUM')
del columbia[-1]
del columbia[0]

In [77]:
def get_name(row):
    first_part = row[12:]
    last = first_part.split(',')[0]
    middle = ''
    for count, x in enumerate(first_part.split(',')[1].split(' ')):
        if (len(re.findall('[A-Z][^A-Z]*',x)) == 1) & (count == 0):
            first = re.findall('[A-Z][^A-Z]*',x)[0]
        if (len(re.findall('[A-Z][^A-Z]*',x)) == 1) & (count > 0):
                middle += f' {x}'
        if (len(re.findall('[A-Z][^A-Z]*',x)) > 1) & (count > 0):
            middle +=  f" {re.findall('[A-Z][^A-Z]*',x)[0]}"
            splitter = first + " " + middle.lstrip()
            break
        if (len(re.findall('[A-Z][^A-Z]*',x)) > 1) & (count == 0):
            first = re.findall('[A-Z][^A-Z]*',x)[0]
            splitter = first
            break
    middle = middle.lstrip()
    return first, middle, last, splitter

In [78]:
def get_department(row, my_split):
    department = ''
    if len(row.split(my_split)[1].split(' ')) == 1:
        department = re.findall('[A-Z][^A-Z]*', row.split(my_split)[1])[0]
    else:
        for counter, x in enumerate(row.split(my_split)[1].split(' ')):
            try:
                int(x[0])
                department += f" {x[0]}"
                break
            except:
                if (len(re.findall('[A-Z][^A-Z]*',x)) == 1):
                    department += f" {re.findall('[A-Z][^A-Z]*',x)[0]}"
                elif (len(re.findall('[A-Z][^A-Z]*',x)) == 0):
                    department += f" {x}"
                elif (len(re.findall('[A-Z][^A-Z]*',x)) > 1) & (counter == 0):
                    department += f" {x}"
                elif (len(re.findall('[A-Z][^A-Z]*',x)) > 1):
                    department += f" {re.findall('[A-Z][^A-Z]*',x)[0]}"
                    break
    department = department.strip()
    return department

In [79]:
def get_title_rate(row, my_split):
    splitt = get_department(row, my_split)
    title = row.split(splitt)[1].split('$')[0]
    rate = row.split(splitt)[1].split('$')[1]
    return title, rate

In [80]:
def clean_row(row):
    first_name, middle_name, last_name, splitter = get_name(row)
    department = get_department(row, splitter)
    title, rate = get_title_rate(row, splitter)
    return (first_name, last_name, middle_name, department, title, rate)

In [81]:
f_name = []
l_name = []
m_name = []
dep = []
title_list = []
rate_list = []
probs = []
for counter, x in enumerate(columbia):
    if counter % 250 == 0:
        print(f'Row {counter} out of {len(columbia)}')
    try:
        fn, ln, mn, dpt, ttl, rt = clean_row(x)
        f_name.append(fn)
        l_name.append(ln)
        m_name.append(mn)
        dep.append(dpt)
        title_list.append(ttl)
        rate_list.append(rt)
    except:
        probs.append(counter)

Row 0 out of 17136
Row 250 out of 17136
Row 500 out of 17136
Row 750 out of 17136
Row 1000 out of 17136
Row 1250 out of 17136
Row 1500 out of 17136
Row 1750 out of 17136
Row 2000 out of 17136
Row 2250 out of 17136
Row 2500 out of 17136
Row 2750 out of 17136
Row 3000 out of 17136
Row 3250 out of 17136
Row 3500 out of 17136
Row 3750 out of 17136
Row 4000 out of 17136
Row 4250 out of 17136
Row 4500 out of 17136
Row 4750 out of 17136
Row 5000 out of 17136
Row 5250 out of 17136
Row 5500 out of 17136
Row 5750 out of 17136
Row 6000 out of 17136
Row 6250 out of 17136
Row 6500 out of 17136
Row 6750 out of 17136
Row 7000 out of 17136
Row 7250 out of 17136
Row 7500 out of 17136
Row 7750 out of 17136
Row 8000 out of 17136
Row 8250 out of 17136
Row 8500 out of 17136
Row 8750 out of 17136
Row 9000 out of 17136
Row 9250 out of 17136
Row 9500 out of 17136
Row 9750 out of 17136
Row 10000 out of 17136
Row 10250 out of 17136
Row 10500 out of 17136
Row 10750 out of 17136
Row 11000 out of 17136
Row 11250 o

In [82]:
# Grab last row from Columbia 
last_col = last[:55]
fn, ln, mn, dpt, ttl, rt = clean_row(last_col)
f_name.append(fn)
l_name.append(ln)
m_name.append(mn)
dep.append(dpt)
title_list.append(ttl)
rate_list.append(rt)

In [83]:
df_columbia = pd.DataFrame({'First_Name': f_name,
                           'Middle_Name': m_name,
                           'Last_name': l_name,
                           'Department': dep,
                           'Job_Title': title_list,
                           'Rate': rate_list})

In [84]:
df_columbia['Location'] = 'COLUM/HOSPT/UMSYS'

In [85]:
df_columbia.head()

,First_Name,Middle_Name,Last_name,Department,Job_Title,Rate,Location
0,Marta,,Abadi,Staffing Support Services,"NURSE, STAFF FLT POOL",42.00,COLUM/HOSPT/UMSYS
1,Lysander,Orbita,Abadia,Clinical Simulation Center,TEACHING AST,13.00,COLUM/HOSPT/UMSYS
2,Erin,Rochelle,Abadir,School of Natural Resources,RESEARCH/LAB TECHNICIAN SR,16.45,COLUM/HOSPT/UMSYS
3,Nilvie,O,Abayan,Sterile Processing,SUPV STERILE PROCESSING,25.28,COLUM/HOSPT/UMSYS
4,Emily,Elizabeth,Abbene,Psychological Sciences,RESEARCH/LAB TECHNICIAN SR,17.31,COLUM/HOSPT/UMSYS


In [86]:
df_columbia[df_columbia['Last_name'] == 'Stannard']

,First_Name,Middle_Name,Last_name,Department,Job_Title,Rate,Location
14390,James,Patrick,Stannard,Orthopaedic Surgery,PROFESSOR,"911,225",COLUM/HOSPT/UMSYS
14391,Rebecca,Faith Emiko,Stannard,Peds O,PREHAB THERAPY AIDE,14.00,COLUM/HOSPT/UMSYS


In [87]:
# Make KC DataFrame
kc = last[55:].split('KCITY')
del kc[0]
rolla = kc[-1]
del kc[-1]

In [88]:
def get_name_not_columbia(row):
    last = row.split(',')[0]
    middle = ''
    for count, x in enumerate(row.split(',')[1].split(' ')):
        if (len(re.findall('[A-Z][^A-Z]*',x)) == 1) & (count == 0):
            first = re.findall('[A-Z][^A-Z]*',x)[0]
        if (len(re.findall('[A-Z][^A-Z]*',x)) == 1) & (count > 0):
                middle += f' {x}'
        if (len(re.findall('[A-Z][^A-Z]*',x)) > 1) & (count > 0):
            middle +=  f" {re.findall('[A-Z][^A-Z]*',x)[0]}"
            splitter = first + " " + middle.lstrip()
            break
        if (len(re.findall('[A-Z][^A-Z]*',x)) > 1) & (count == 0):
            first = re.findall('[A-Z][^A-Z]*',x)[0]
            splitter = first
            break
    middle = middle.lstrip()
    return first, middle, last, splitter

In [89]:
def clean_row(row):
    first_name, middle_name, last_name, splitter = get_name_not_columbia(row)
    department = get_department(row, splitter)
    title, rate = get_title_rate(row, splitter)
    return (first_name, last_name, middle_name, department, title, rate)

In [90]:
f_name = []
l_name = []
m_name = []
dep = []
title_list = []
rate_list = []
probs = []
for counter, x in enumerate(kc):
    if counter % 250 == 0:
        print(f'Row {counter} out of {len(kc)}')
    try:
        fn, ln, mn, dpt, ttl, rt = clean_row(x)
        f_name.append(fn)
        l_name.append(ln)
        m_name.append(mn)
        dep.append(dpt)
        title_list.append(ttl)
        rate_list.append(rt)
    except:
        probs.append(counter)

Row 0 out of 3175
Row 250 out of 3175
Row 500 out of 3175
Row 750 out of 3175
Row 1000 out of 3175
Row 1250 out of 3175
Row 1500 out of 3175
Row 1750 out of 3175
Row 2000 out of 3175
Row 2250 out of 3175
Row 2500 out of 3175
Row 2750 out of 3175
Row 3000 out of 3175


In [91]:
# Add last row from KC
fn, ln, mn, dpt, ttl, rt = clean_row(rolla[:50])
f_name.append(fn)
l_name.append(ln)
m_name.append(mn)
dep.append(dpt)
title_list.append(ttl)
rate_list.append(rt)

In [92]:
df_kc = pd.DataFrame({'First_Name': f_name,
                           'Middle_Name': m_name,
                           'Last_name': l_name,
                           'Department': dep,
                           'Job_Title': title_list,
                           'Rate': rate_list})

In [93]:
df_kc['Location'] = 'KCITY'

In [94]:
# Make Rolla DataFrame
rolla = rolla[50:].split('ROLLA')
del rolla[0]
stl = rolla[-1]
del rolla[-1]

In [95]:
f_name = []
l_name = []
m_name = []
dep = []
title_list = []
rate_list = []
probs = []
for counter, x in enumerate(rolla):
    if counter % 250 == 0:
        print(f'Row {counter} out of {len(kc)}')
    try:
        fn, ln, mn, dpt, ttl, rt = clean_row(x)
        f_name.append(fn)
        l_name.append(ln)
        m_name.append(mn)
        dep.append(dpt)
        title_list.append(ttl)
        rate_list.append(rt)
    except:
        probs.append(counter)

Row 0 out of 3175
Row 250 out of 3175
Row 500 out of 3175
Row 750 out of 3175
Row 1000 out of 3175
Row 1250 out of 3175


In [96]:
fn, ln, mn, dpt, ttl, rt = clean_row(stl[:90])
f_name.append(fn)
l_name.append(ln)
m_name.append(mn)
dep.append(dpt)
title_list.append(ttl)
rate_list.append(rt)

In [97]:
df_rolla = pd.DataFrame({'First_Name': f_name,
                           'Middle_Name': m_name,
                           'Last_name': l_name,
                           'Department': dep,
                           'Job_Title': title_list,
                           'Rate': rate_list})

In [98]:
df_rolla['Location'] = 'ROLLA'

In [99]:
# Make STL Dataframe
stl = stl[90:].split('STLOU')
del stl[0]

In [100]:
f_name = []
l_name = []
m_name = []
dep = []
title_list = []
rate_list = []
probs = []
for counter, x in enumerate(stl):
    if counter % 250 == 0:
        print(f'Row {counter} out of {len(kc)}')
    try:
        fn, ln, mn, dpt, ttl, rt = clean_row(x)
        f_name.append(fn)
        l_name.append(ln)
        m_name.append(mn)
        dep.append(dpt)
        title_list.append(ttl)
        rate_list.append(rt)
    except:
        probs.append(counter)

Row 0 out of 3175
Row 250 out of 3175
Row 500 out of 3175
Row 750 out of 3175
Row 1000 out of 3175
Row 1250 out of 3175
Row 1500 out of 3175
Row 1750 out of 3175


In [101]:
df_stl = pd.DataFrame({'First_Name': f_name,
                           'Middle_Name': m_name,
                           'Last_name': l_name,
                           'Department': dep,
                           'Job_Title': title_list,
                           'Rate': rate_list})

In [102]:
df_stl['Location'] = 'STLOU'

In [103]:
df_columbia.to_csv('./data/columbia.csv', index = False)
df_kc.to_csv('./data/kc.csv', index = False)
df_rolla.to_csv('./data/rolla.csv', index = False)
df_stl.to_csv('./data/stl.csv', index = False)

In [104]:
df_columbia.shape, df_kc.shape, df_rolla.shape, df_stl.shape

((17069, 7), (3141, 7), (1495, 7), (1876, 7))

In [105]:
# make error dataframe

In [106]:
location = ['Columbia', 'KC', 'Rolla', 'STL']
actual_rows = [df_columbia.shape[0], df_kc.shape[0], df_rolla.shape[0], df_stl.shape[0]]
last_row = 1
expected_rows = [len(columbia) + last_row, len(kc) + last_row, len(rolla) + last_row, len(stl)]

In [107]:
error_df = pd.DataFrame({'Location': location,
                        'actual_rows': actual_rows,
                        'expected_rows': expected_rows})

In [108]:
error_df['Percentage_of_Actual_Rows'] = error_df['actual_rows'] / error_df['expected_rows']

In [109]:
error_df

,Location,actual_rows,expected_rows,Percentage_of_Actual_Rows
0,Columbia,17069,17137,0.996032
1,KC,3141,3176,0.988980
2,Rolla,1495,1496,0.999332
3,STL,1876,1879,0.998403


In [110]:
error_df['actual_rows'].sum() / error_df['expected_rows'].sum() 

0.9954829449510301